## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import datetime

In [2]:
now=datetime.datetime.now()
hour=now.hour

## Fetching Top Gainers Of The Day

In [6]:
def top_gainers():
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://www.nseindia.com/market-data/top-gainers-losers',
    })
    try:
        main_page = session.get('https://www.nseindia.com/market-data/top-gainers-losers', timeout=10)
        api_url = "https://www.nseindia.com/api/live-analysis-variations?index=gainers"
        response = session.get(api_url, timeout=10)
        response.raise_for_status()
        data = response.json()['allSec']
        if 'data' in data:
            df = pd.DataFrame(data['data'])
            return df
        else:
            print("Unexpected response structure:", data.keys())
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
# if (hour<12 and hour>9):
#     df1 = top_gainers()
# else:
#     print('Time Lapsed')
df1 = top_gainers()

## Stocks at 20% Upper Circuit [Change >= 19.90% eqivalent to 20% as per NSE]

In [7]:
df=df1.copy()
df=df[['symbol','net_price','ltp']]
df = df[~df['symbol'].str.endswith('RE')]
try:
    df=(df[df['net_price']>=19.90]).drop(columns='net_price')
except:
    print('No stock hit 20% upper circuit')
# df.index=range(1,len(df)+1)
# df=df.rename_axis('S.No.')

In [8]:
if not df.empty:
    df.to_csv('Upper Circuit.csv',mode='a',header=False,index=False)

## Checking Order Book & Exit Price

In [12]:
def orderbook(stock):
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': f'https://www.nseindia.com/get-quotes/equity?symbol={stock}',
    })
    try:
        main_page = session.get(f'https://www.nseindia.com/get-quotes/equity?symbol={stock}', timeout=10)
        api_url = f'https://www.nseindia.com/api/quote-equity?symbol={stock}&section=trade_info'
        response = session.get(api_url, timeout=10)
        response.raise_for_status()
        data = response.json()['marketDeptOrderBook']['totalSellQuantity']
        if data!=0:
            data2=response.json()['marketDeptOrderBook']['ask'][0]['price']
            return data2
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

## Exiting Trade & Finding PnL (%)

In [ ]:
temp=pd.read_csv("Upper Circuit.csv")
lst=list(temp[temp['Sell Price'].isnull()]['Symbol'])
for i in range(len(lst)):
    check=orderbook(lst[i])
    if check:
        temp.loc[temp['Symbol']==lst[i],'Sell Price']=check
        temp.loc[temp['Symbol']==lst[i],'PnL (%)']=round((check-temp.loc[temp['Symbol']==lst[i],'Buy Price'])*100/temp.loc[temp['Symbol']==lst[i],'Buy Price'],2)

In [ ]:
temp.to_csv('Upper Circuit.csv',mode='w',header=True,index=False)

## Fetching Next Day's Open Price (Sell Price)

In [3]:
def open(stock):
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': f'https://www.nseindia.com/get-quotes/equity?symbol={stock}',
    })
    try:
        main_page = session.get(f'https://www.nseindia.com/get-quotes/equity?symbol={stock}', timeout=10)
        api_url = f'https://www.nseindia.com/api/quote-equity?symbol={stock}'
        response = session.get(api_url, timeout=10)
        response.raise_for_status()
        data = response.json()['priceInfo']['open']
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

## Holding Trade PnL (%)

In [4]:
df2=pd.read_csv("Upper Circuit.csv")
lst=list(df2[df2['Sell Price'].isnull()]['Symbol'])
for i in range(len(lst)):
    result=open(lst[i])
    df2.loc[df2['Symbol']==lst[i],'Sell Price']=result
    df2.loc[df2['Symbol']==lst[i],'PnL (%)']=round((result-df2.loc[df2['Symbol']==lst[i],'Buy Price'])*100/df2.loc[df2['Symbol']==lst[i],'Buy Price'],2)

In [5]:
df2.to_csv('Upper Circuit.csv',mode='w',header=True,index=False)